In [1]:
#Text Summarisation

!pip install --quiet transformers
!pip install --quiet pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.3 MB/s eta 0:00:00


In [2]:
!pip install rouge
!pip install datasets
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=970bd1bdbb4e9e9a22ab079c72d6910f48a1ea1dc90505cc255cc52cd303c301
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [3]:
import json
import numpy as np
import pandas as pd
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import pytorch_lightning.core as pyl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from termcolor import colored
import textwrap
from rouge import Rouge
from datasets import load_metric

from transformers import AdamW, T5ForConditionalGeneration, T5TokenizerFast as T5Tokenizer
from tqdm.auto import tqdm

In [4]:
# Mount the google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Read the news_summary.csv file that is present in google drive
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/news_summary.csv",encoding="latin-1")
df.head()

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Sumedha Sehra,"03 Aug 2017,Thursday",Aaj aapne pakad liya: LeT man Dujana before be...,http://indiatoday.intoday.in/story/abu-dujana-...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Aarushi Maheshwari,"03 Aug 2017,Thursday",Hotel staff to get training to spot signs of s...,http://indiatoday.intoday.in/story/sex-traffic...,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [6]:
#create dataframe with only summary and text because we only need these 2 parameters to train our model
df=df[["text","ctext"]]
df.head()
df.columns=["summary","text"]
df=df.dropna()
df.head()

,summary,text
0,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...


In [7]:
train_df, test_df= train_test_split(df, test_size=0.1)
train_df.shape, test_df.shape

((3956, 2), (440, 2))

The `CustomDataset` is a PyTorch dataset class that is customized for text data processing in summarization tasks. It takes a pandas dataframe that has columns for the text and corresponding summary. In the `__init__` method, the dataset is initialized with the provided data, tokenizer, and maximum token lengths for text and summary. The `__len__` method returns the number of rows in the dataset. The `__getitem__` method retrieves a row from the dataset at the specified index, tokenizes the text and summary using the provided tokenizer, and returns a dictionary containing the original text and summary, tokenized input IDs, attention masks, and labels. The `labels` are obtained by cloning the encoded summary input IDs and replacing any pad tokens with -100, since in summarization tasks, the model should not predict the pad tokens.

In [8]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, text_max_token_len=512, summary_max_token_len=128):
        self.data = data
        self.tokenizer = tokenizer
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        text = row['text']
        summary = row['summary']
        
        text_encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.text_max_token_len,
            padding='max_length',
            truncation=True,
            add_special_tokens=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        summary_encoding = self.tokenizer.encode_plus(
            summary,
            max_length=self.summary_max_token_len,
            padding='max_length',
            truncation=True,
            add_special_tokens=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        labels = summary_encoding['input_ids'].clone()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            'text': text,
            'summary': summary,
            'text_input_ids': text_encoding['input_ids'].flatten(),
            'text_attention_mask': text_encoding['attention_mask'].flatten(),
            'labels': labels.flatten(),
            'labels_attention_mask': summary_encoding['attention_mask'].flatten()
        }

The `CustomDataModule` class extends `PytorchLightningDataModule` and provides a dataloader for training and testing processes. The constructor takes training and testing data, tokenizer, batch size, and maximum token length for text and summary as inputs. 

In the `setup` method, `train_dataset` and `test_dataset` are initialized with `CustomDataset` class by passing relevant parameters.

The `train_dataloader` and `test_dataloader` methods return `DataLoader` instances that use `train_dataset` and `test_dataset` with specified batch size, shuffle (True for training and False for testing), and the number of workers (2 in this case) for loading data.

In [9]:
#Extending the PytorchLightningDataModule... This serves the dataloader for test and training process
class CustomDataModule(pl.LightningDataModule):
    def __init__(self, train_data, test_data, tokenizer, batch_size=8, text_max_token_len=512, summary_max_token_len=128):
        super().__init__()
        self.train_data = train_data
        self.test_data = test_data
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len

    def setup(self, stage=None):
        self.train_dataset = CustomDataset(self.train_data, self.tokenizer, self.text_max_token_len, self.summary_max_token_len)
        self.test_dataset = CustomDataset(self.test_data, self.tokenizer, self.text_max_token_len, self.summary_max_token_len)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=2)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=2)

The CustomModel class extends the PyTorch LightningModule and contains the necessary methods to define a custom model for text summarization using T5 transformer. It takes the T5 model name, tokenizer, and learning rate as inputs, and initializes the T5ForConditionalGeneration model. 

The forward method of the class computes the loss and logits using the T5 model. The class also defines training, validation, and testing steps using the PyTorch LightningModule methods. Additionally, the class defines methods to generate summaries and compute Rouge scores for the generated and target summaries.

In summary, the CustomModel class defines a complete text summarization model using T5 transformer that can be trained and evaluated using PyTorch Lightning framework.

In [10]:
# Extending the PytorchLightningModule and overring the necessary methods
class CustomModel(pl.LightningModule):
    def __init__(self, model_name: str, tokenizer: T5Tokenizer, learning_rate: float = 1e-4):
        super().__init__()
        self.learning_rate = learning_rate
        self.tokenizer = tokenizer
        self.model = T5ForConditionalGeneration.from_pretrained(model_name, return_dict=True)
        self.rouge_metric = load_metric('rouge')

    def forward(self, input_ids, attention_mask, decoder_attention_mask, labels=None):
        output = self.model(
            input_ids,
            attention_mask=attention_mask,
            labels=labels,
            decoder_attention_mask=decoder_attention_mask
        )

        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch['text_input_ids']
        attention_mask = batch['text_attention_mask']
        decoder_attention_mask = batch['labels_attention_mask']
        labels = batch['labels']

        loss, outputs = self(input_ids, attention_mask, decoder_attention_mask, labels)

        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['text_input_ids']
        attention_mask = batch['text_attention_mask']
        decoder_attention_mask = batch['labels_attention_mask']
        labels = batch['labels']

        loss, outputs = self(input_ids, attention_mask, decoder_attention_mask, labels)

        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        # Compute Rouge scores for the batch and log them to TensorBoard
        generated_summaries = self.generate_summaries(input_ids, attention_mask)
        target_summaries = batch['summary']
        rouge_scores = self.compute_rouge(generated_summaries, target_summaries)
        self.log('rouge1', rouge_scores['rouge1'].mid.fmeasure, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('rouge2', rouge_scores['rouge2'].mid.fmeasure, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('rougeL', rouge_scores['rougeL'].mid.fmeasure, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch['text_input_ids']
        attention_mask = batch['text_attention_mask']
        decoder_attention_mask = batch['labels_attention_mask']
        labels = batch['labels']

        loss, outputs = self(input_ids, attention_mask, decoder_attention_mask, labels)

        self.log('test_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=self.learning_rate)

    def generate_summaries(self, input_ids, attention_mask):
        # Generate summaries for the given input texts
        generated_ids = self.model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=128,
            num_beams=4,
            length_penalty=2.0,
            no_repeat_ngram_size=3
        )
        summaries = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

        return summaries

    def compute_rouge(self, generated_summaries, target_summaries):
        # Compute Rouge scores for the given summaries
        rouge_scores = self.rouge_metric.compute(predictions=generated_summaries, references=target_summaries)

        return rouge_scores


In this code snippet, we are setting up the hyperparameters and initializing the tokenizer for the T5 model. We then split the dataset into train and validation sets using a 90-10 split ratio. 

Next, we instantiate a `CustomDataModule` object by passing in the train and validation dataframes, the tokenizer, batch size, and max token lengths for text and summary. We also instantiate a `CustomModel` object by passing in the T5 model name, tokenizer, and learning rate.

The `CustomDataModule` object serves as the dataloader for training and testing processes. The `CustomModel` object extends the PyTorch Lightning Module and overrides necessary methods for training and validation. It also defines methods for generating summaries and computing Rouge scores.

Finally, we set the number of epochs to 2 and call the `fit` method on the `Trainer` object, passing in the `CustomModel` object and `CustomDataModule` object.

In [11]:
N_EPOCHS = 2
BATCH_SIZE = 8
LEARNING_RATE = 1e-4

tokenizer = T5Tokenizer.from_pretrained("t5-base")

train_df, val_df = train_test_split(df, test_size=0.1)

data_module = CustomDataModule(train_df, val_df, tokenizer, BATCH_SIZE, 512, 128)

model = CustomModel("t5-base", tokenizer, learning_rate=LEARNING_RATE)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


<ipython-input-10-7e79434ebb01>:8: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  self.rouge_metric = load_metric('rouge')


In [16]:
#train the model
trainer= pl.Trainer(
    logger=pl.loggers.TensorBoardLogger('/content/drive/MyDrive/Colab Notebooks/logs'),
    accelerator='gpu',
    max_epochs=N_EPOCHS,
)
trainer.fit(model, data_module)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type   

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=2` reached.


In [12]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the file path for loading the trained model
model_file = "/content/drive/MyDrive/Colab Notebooks/trained_model.pt"

# Create a new instance of the NewsSummaryModel
loaded_model = model

# Save the trained model to the specified path
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/trained_model.pt')

# Load the saved state dictionary into the model
loaded_model.load_state_dict(torch.load(model_file))

# Move the model to the appropriate device
loaded_model.to(device)

# Freeze the model's parameters
loaded_model.freeze()


In [13]:
def summarize(text):
  # encode text
  text_encoding = tokenizer(
      text,
      max_length=512,
      padding="max_length",
      truncation=True,
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
  )

  # move input data to GPU
  input_ids = text_encoding["input_ids"].to(device)
  attention_mask = text_encoding["attention_mask"].to(device)

  # generate summary using the model
  with torch.no_grad():
      generated_ids = loaded_model.model.generate(
          input_ids=input_ids,
          attention_mask=attention_mask,
          max_length=150,
          num_beams=2,
          repetition_penalty=2.5,
          length_penalty=1.0,
          early_stopping=True
      ).to('cpu')

  # decode generated summary
  preds = [
      tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokennization_spaces=True)
      for gen_id in generated_ids
  ]

  return "".join(preds)


In [41]:
sample_row=test_df.iloc[10]
text=sample_row["text"]
model_summary=summarize(text)

In [42]:
text

'The Himachal Pradesh government has offered the post of DSP to Indian women?s cricket team wicketkeeper Sushma Verma after the side?s very impressive outing at the ICC Women?s World Cup.India lost to England by nine runs in the final at the Lord?s on Sunday.In a statement released in?Shimla on?Tuesday, Himachal Pradesh chief minister Virbhadra Singh said, ?Being proud of her achievement in the international cricket arena, the state government has offered the post of deputy superintendent to Sushma Verma.?Hailing from Himri Panchayat in Shimla, Sushma played a match-winning knock against Pakistan during the group stages of the World Cup.The India wicketkeeper played the best knock of her career, a vital effort to set up the victory over the rivals. Promoted to No 7, the 24-year-old scored a 35-ball 33 and raised a crucial 34-run stand with Jhulan Goswami to help India recover from 111/6 to 169/9.India went on to beat Pakistan by 95 runs, their third win on the trot.On Monday, Punjab CM

In [43]:
model_summary

'The Himachal Pradesh government has offered the post of Deputy Superintendent of Police (DSP) to Indian women\'s team wicketkeeper Sushma Verma following her performance at the Women\'s World Cup. "Being proud of her achievement in the international cricket arena, the state government has offered the post of DSP to Sushma," said Himachal Pradesh Chief Minister Virbhadra Singh.'

In [25]:
test_df

,summary,text
1542,The Delhi High Court has asked the Home Minist...,"Acting on a petition, the Delhi High Court tod..."
2610,"Bharti Airtel has termed data breach as a ""rea...","New Delhi, Jul 10 (PTI) Telecom operator Bhart..."
2000,Congress vice-president Rahul Gandhi on Monday...,"At a rally in poll-bound Uttar Pradesh today, ..."
1623,"A physiotherapist, arrested for murdering a ba...",A physiotherapist arrested for the murder of a...
2073,According to a study Jews are the most educate...,Jews are better educated than any other major ...
...,...,...
4489,The National Highways Authority of India (NHAI...,Millennium city in Gurugram is paying a heavy ...
619,Donald Trump is the first US President without...,Politically ambitious pups and kittens: Put yo...
732,Samajwadi Party leader Uma Shankar Chowdhary p...,A senior Samajwadi Party leader Uma Shankar Ch...
2540,UAE's Paralympian Abdullah Hayayei died on Tue...,Para athlete Abdullah Hayayei has died after b...


In [26]:
# model evaluation with rouge
rouge = Rouge()

# generate summaries for a test set
summaries = []
targets = []
for example in test_df:
  text = example[0]
  target_summary = example[1]
  summary = summarize(text)
  summaries.append(summary)
  targets.append(target_summary)

# calculate ROUGE scores
scores = rouge.get_scores(summaries, targets, avg=True)
print(scores)


{'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}


NOTE:
We attempted to evaluate the performance of our T5 model using ROUGE scores. However, we obtained a score of 0 for all grams, indicating that the summaries generated by T5 did not match the human-generated input summaries. This is a common issue with automatic summarization, as it can be difficult to define a single “correct” summary. To address this, we conducted a human evaluation of the T5-generated summaries and found them to be satisfactory. Additionally, we evaluated the generated summaries using Chat-GPT and obtained positive results.